## Creating Archetypes (New API)

> Related reading
> - ../index.md
> - P3O tutorial: ../optimizing-on-prompts/index.md

#### Step 1: Setup
First, let's set up our environment and import the necessary libraries:

In [ ]:
from agent_torch.core.llm.archetype import Archetype
from agent_torch.core.llm.mock_llm import MockLLM
import agent_torch.populations.astoria as astoria
import pandas as pd, os

#### Step 2: Initialise LLM Instance

Use any LLM, as long as it implements the required __call__(prompt_inputs) hook.

- Required: __call__(prompt_inputs: list[{"agent_query": str, "chat_history": list}]) -> list[str|{"text": str}]
- Optional: initialize_llm(self) for one-time setup
- Output must be float convertible, view index.md for more details.

AgentTorch supplies per-agent chat_history and parses outputs into floats.

Example minimal implementation for GPT 3.5 Turbo

In [ ]:
from openai import OpenAI
import os

class ChatGPT35LLM:
    def __init__(self, api_key=None, model="gpt-3.5-turbo", temperature=0.2):
        self.api_key = api_key or os.getenv("OPENAI_API_KEY")
        self.model = model
        self.temperature = temperature
        self.client = None

    def initialize_llm(self):
        self.client = OpenAI(api_key=self.api_key)
        return self

    def __call__(self, prompt_inputs):
        client = self.client or OpenAI(api_key=self.api_key)
        outputs = []
        for item in prompt_inputs:
            content = str(item.get("agent_query", ""))  # already includes system instruction
            resp = client.chat.completions.create(
                model=self.model,
                temperature=self.temperature,
                messages=[{"role": "user", "content": content}],
            )
            text = (resp.choices[0].message.content or "").strip()
            outputs.append({"text": text})
        return outputs

#### Step 3: Define an Archetype

In [ ]:
# Step 3: Define Variables and a Template
import agent_torch.core.llm.template as lm

class MyPromptTemplate(lm.Template):
    system_prompt = "Consider the following variables and determine your willingness to isolate."
    # Variables (choose which are learnable)
    salary = lm.Variable(desc="salary")
    work_week = lm.Variable(desc="hours worked in a week", learnable=True)
    satisfaction = lm.Variable(desc="categorical satisfaction levels", learnable=False, default="unknown satisfaction")
    def __prompt__(self):
        self.prompt_string = (
            "I work for {work_week} hours, and get paid ${salary}. "
            "I am {satisfaction} with my current job."
        )
    def __output__(self):
        return "Answer on a scale from 0 - 1.0."

prompt_template = MyPromptTemplate()
arch = Archetype(prompt=prompt_template, llm=llm, n_arch=3)

Sample Data for Variable objects:

In [ ]:
# Step 3: Configure external data and preview
import pandas as pd

# Example dataframe (from index.md)
df = pd.DataFrame({
    "Salary": [65000, 43000, 120000],
    "Work_Week_Hours": [60, 40, 80],
    "Satisfaction_Level": ["Not satisfied", "Partially satisfied", "Very satisfied"],
})


#### Step 4: Compare performance between different Configurations of Archetype

In [ ]:
# Step 4: Broadcast to a population and sample decisions
arch.configure(external_df=df, split = 2) #configure with only 2 rows
arch.sample()

from agent_torch.populations import astoria
arch.configure(external_df=df) # configure with full dataframe
arch.broadcast(population=astoria) # bind population
arch.sample() # returns (n_agents,) tensor of decisions